## Neural Nets
You  must also try to do digit recognition using neural nets, in particular in Googles Open Source Framework, Tensorflow. Read the guide the TA gratiously has provided for you. 

### Deliverables
You must try using a classific neural net with one input layer, one hidden layer, and one output layer.
You can chosse whichever non-linear activation function you want but we suggest using logistic or rectified linear unit (relu).  You must again experiment with the hyperparameters, in particular
network size (hidden layer) and regularization (weight decay, droupout), and use
validation to find the best combination for the digits data. The validation data should come from the training data not the test data.

## Report
In the report, explain what you have done and and provide
tables that show your results. Include tables of your validation
results for all the SVMs and Neural Nets you have tested as well as the out 
of sample error/accuracy computed using the test set for the best SVM and the neural net you have found so far.


In [6]:

%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
auTrain = np.load('auTrain.npz')
auTrain_images = auTrain['digits']
auTrain_labels = auTrain['labels']
auTest = np.load('auTest.npz')
auTest_images = auTest['digits']
auTest_labels = auTest['labels']

d = auTrain_images.shape[1]

def plot_perf(names, fig=None, ax=None, xlim=None, ylim=None, figsize=(15, 10)):
    import matplotlib
    from IPython.display import display, clear_output

    formats = ['%s: %%s' % n for n in names]
    n = len(formats)

    if ax is None or isinstance(ax, int):
        if fig is None:
            fig = matplotlib.figure.Figure(figsize=figsize)
            fig.canvas = matplotlib.backends.backend_agg.FigureCanvasAgg(fig)

        if isinstance(ax, int):
            ax = fig.add_subplot(ax)
        else:
            ax = fig.gca()
        if xlim is None:
            ax.set_autoscalex_on(True)
        else:
            ax.set_autoscalex_on(False)
            ax.set_xlim(*xlim)
        if ylim is None:
            ax.set_autoscaley_on(True)
        else:
            ax.set_autoscaley_on(False)
            ax.set_ylim(*ylim)
        ax.grid()
    lines = [([], [], ax.plot([], [])[0]) for _ in range(n)]

    def add(*points):
        legend = ax.get_legend()
        if legend is not None:
            legend.remove()

        for y, (xs, ys, line) in zip(points, lines):
            xs.append(len(xs))
            ys.append(y)
            line.set_data(xs, ys)
        ax.legend([f % p for f, p in zip(formats, points)])
        ax.relim()
        ax.autoscale(enable=None)
        clear_output(wait=True)
        display(fig)

    return fig, add

def permute_data_all():
    assert auTrain_images.flags.c_contiguous
    # Return a random permutation of the data
    perm = np.random.permutation(len(auTrain_labels))
    return auTrain_images[perm], auTrain_labels[perm]

def neural_network(X_train, y_train, X_test, y_test, n_layers = 100, prob = 0.9, epochs = 500, batch_size = 16, reg_rate = 1e-4, plotting = True):
    starttid =datetime.datetime.now().time()
    auTrain_images = X_train
    auTrain_labels = y_train
    auTest_images = X_test
    auTest_labels = y_test
    with tf.Graph().as_default():
        
        data = tf.placeholder(tf.float32, shape = [None, 784])
        labels = tf.placeholder(tf.int32, shape = [None])
        learning_rate = tf.placeholder(tf.float32)
        #layer 1
        W1 = tf.Variable(tf.truncated_normal([784, n_layers], stddev=0.1))
        b1 = tf.Variable(tf.truncated_normal([n_layers], stddev=0.1))
        y1 = tf.nn.relu(tf.matmul(data, W1) + b1)
        
        #dropoutlayer
        keep_prob = tf.placeholder(tf.float32)
        drop = tf.nn.dropout(y1, keep_prob)

        #Output
        W2 = tf.Variable(tf.truncated_normal([n_layers, 10], stddev=0.1))
        b2 = tf.Variable(tf.truncated_normal([10], stddev=0.1))
        y2 = tf.nn.softmax(tf.matmul(drop, W2) + b2)

        correct_prediction = tf.equal(labels, tf.cast(tf.argmax(y2, 1), labels.dtype))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        y2, labels)

        
        loss = tf.reduce_mean(cross_entropy)
        reg = reg_rate * (tf.reduce_sum(W2 ** 2)+tf.reduce_sum(W1 ** 2)+tf.reduce_sum(b1 ** 2)+tf.reduce_sum(b2 ** 2))
        # Minimization target is the sum of cross-entropy loss and regularization
        target = loss + reg

        opt = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
        train_step = opt.minimize(target)
        fig, perf = plot_perf(names=['E_in', 'loss', 'E_test', 'reg'],
                          ax=211, xlim=(0, epochs-1), ylim=(0, 0.7), figsize=(12, 10))
        
        
        
        with tf.Session() as session:
            session.run(tf.initialize_all_variables())

            for epoch in range(epochs):
                input_value, labels_value = permute_data_all()
    
                # Run train_step in mini-batches
                for i in range(0, len(auTrain_labels), batch_size):
                    j = min(len(auTrain_labels), i+batch_size)
                    results = session.run(
                        train_step,
                        feed_dict={data: input_value[i:j],
                                   labels: labels_value[i:j], keep_prob: prob,
                                   learning_rate: 0.1})

                train_accuracy = session.run(accuracy,feed_dict={data: auTrain_images, labels: auTrain_labels, keep_prob: 1.0})
                correctly_labeled = session.run(correct_prediction, feed_dict={data: auTest_images, labels: auTest_labels, keep_prob: 1.0})
                #print(correctly_labeled)
                test_accuracy = session.run(accuracy, feed_dict={data: auTest_images, labels: auTest_labels, keep_prob: 1.0})
                test_error = 1 - test_accuracy
                #print("test_error ",test_error)
                current_loss = session.run(loss, feed_dict={data: auTrain_images, labels: auTrain_labels, keep_prob: 1.0})
                in_sample_error = 1 - train_accuracy
                if plotting == True:
                    perf(in_sample_error,  current_loss/3, test_error, session.run(reg)/3)

            test_accuracy = session.run( accuracy,
            feed_dict={data: auTest_images,
                       labels: auTest_labels, keep_prob: 1.0})
            test_error = 1 - test_accuracy
            #print(test_error)
            #print("starttid ",starttid," sluttid ",datetime.datetime.now().time())           
            return(test_error)
            
def CV_nn(X_train, y_train, n_layers_list = [10,100,200,300], prob_list = [0.5,0.7,0.9], reg_rate_list = [0.001,1e-4,1e-5]):
    split = ShuffleSplit(n_splits=5, test_size=0.2)
    print(datetime.datetime.now().time())
    res_list = []
    best_error = 1.0
    f=open('neuralresult.txt','w')
    f.close()
    f=open('neuralresult.txt','a')
    for n_layers in n_layers_list:
        for prob in prob_list:
            for reg_rate in reg_rate_list:
                f.write('n_layers: '+str(n_layers)+ ' prob: '+str(prob)+' reg_rate: '+ str(reg_rate)+"\n")
                print('n_layers:', n_layers, 'prob:', prob, 'reg_rate:', reg_rate)
                class_list = []
                for train_index, test_index in split.split(X_train):
                    class_list.append(neural_network(X_train[train_index], y_train[train_index],
                                                 X_train[test_index], y_train[test_index],
                                                     n_layers = n_layers, prob = prob, epochs = 50,
                                                     batch_size = 16, reg_rate = reg_rate, plotting = False))
                mean_error = np.mean(class_list)
                f.write('mean error '+str(mean_error)+"\n")
                print('mean error', mean_error)
                res_dict = {'n_layers' : n_layers, 'prob' : prob, 'reg_rate':reg_rate, 'error' : mean_error}
                res_list.append(res_dict)
                if mean_error < best_error:
                    best_error = mean_error
                    best_params = res_dict

    print("best params",best_params) 
    f.write(str(best_params['n_layers']))
    f.write("\n")
    f.write(str(best_params['prob']))
    f.write("\n")
    f.write(str(best_params['reg_rate']))
    f.write("\n")
    f.write(str(best_params['error']))
    f.write("\n")
    resulterror=neural_network(X_train, y_train,
                               auTest_images, auTest_labels,
                               n_layers = best_params['n_layers'], prob = best_params['prob'], epochs = 50,
                               batch_size = 16, reg_rate = best_params['reg_rate'], plotting = False)
    f.write("test result ")
    f.write(str(resulterror))
    f.write('\n')
    f.close()
    return best_params,resulterror
if __name__ == '__main__':
    #print(neural_network(auTrain_images, auTrain_labels, auTest_images, auTest_labels))
    print (CV_nn(auTrain_images, auTrain_labels))
    print("sluttid ",datetime.datetime.now().time())

10:20:35.978523
n_layers: 300 prob: 0.9 reg_rate: 0.0001
mean error 0.01348747015
n_layers: 300 prob: 0.9 reg_rate: 1e-05
mean error 0.0328516364098
best params {'n_layers': 300, 'prob': 0.9, 'error': 0.013487470149993897, 'reg_rate': 0.0001}
({'n_layers': 300, 'prob': 0.9, 'error': 0.013487470149993897, 'reg_rate': 0.0001}, 0.044186055660247803)
sluttid  11:49:24.564854


In [ ]:
11:13:58.371865
n_layers: 10 prob: 0.5 reg_rate: 1e-05
mean error 0.162909448147
n_layers: 10 prob: 0.5 reg_rate: 0.0001
mean error 0.147591531277
n_layers: 10 prob: 0.5 reg_rate: 0.001
mean error 0.136994206905
n_layers: 10 prob: 0.7 reg_rate: 1e-05
mean error 0.106069374084
n_layers: 10 prob: 0.7 reg_rate: 0.0001
mean error 0.102793836594
n_layers: 10 prob: 0.7 reg_rate: 0.001
mean error 0.127841997147
n_layers: 10 prob: 0.9 reg_rate: 1e-05
mean error 0.110789990425
n_layers: 10 prob: 0.9 reg_rate: 0.0001
mean error 0.0780346870422
n_layers: 10 prob: 0.9 reg_rate: 0.001
mean error 0.0989402770996
n_layers: 100 prob: 0.5 reg_rate: 1e-05
mean error 0.0438342928886
n_layers: 100 prob: 0.5 reg_rate: 0.0001
mean error 0.0476878643036
n_layers: 100 prob: 0.5 reg_rate: 0.001
mean error 0.0735067486763
n_layers: 100 prob: 0.7 reg_rate: 1e-05
mean error 0.0459537625313
n_layers: 100 prob: 0.7 reg_rate: 0.0001
mean error 0.0290944337845
n_layers: 100 prob: 0.7 reg_rate: 0.001
mean error 0.0626204371452
n_layers: 100 prob: 0.9 reg_rate: 1e-05
mean error 0.0409441232681
n_layers: 100 prob: 0.9 reg_rate: 0.0001
mean error 0.0214836359024
n_layers: 100 prob: 0.9 reg_rate: 0.001
mean error 0.0514451026917
n_layers: 200 prob: 0.5 reg_rate: 1e-05
mean error 0.0318882465363
n_layers: 200 prob: 0.5 reg_rate: 0.0001
mean error 0.0304431796074
n_layers: 200 prob: 0.5 reg_rate: 0.001
mean error 0.0629094362259
n_layers: 200 prob: 0.7 reg_rate: 1e-05
mean error 0.0198458552361
n_layers: 200 prob: 0.7 reg_rate: 0.0001
mean error 0.019267821312
n_layers: 200 prob: 0.7 reg_rate: 0.001
mean error 0.05838149786
n_layers: 200 prob: 0.9 reg_rate: 1e-05
mean error 0.0164739847183
n_layers: 200 prob: 0.9 reg_rate: 0.0001
mean error 0.0158959388733
n_layers: 200 prob: 0.9 reg_rate: 0.001
mean error 0.0506743907928
n_layers: 300 prob: 0.5 reg_rate: 1e-05
mean error 0.0224470019341
n_layers: 300 prob: 0.5 reg_rate: 0.0001
mean error 0.0439306378365
n_layers: 300 prob: 0.5 reg_rate: 0.001
mean error 0.0622350692749
n_layers: 300 prob: 0.7 reg_rate: 1e-05
mean error 0.0354527950287
n_layers: 300 prob: 0.7 reg_rate: 0.0001
mean error 0.0586705207825
n_layers: 300 prob: 0.7 reg_rate: 0.001
mean error 0.0526011705399
n_layers: 300 prob: 0.9 reg_rate: 1e-05
mean error 0.0328516364098
n_layers: 300 prob: 0.9 reg_rate: 0.0001
mean error 0.01348747015
n_layers: 300 prob: 0.9 reg_rate: 0.001
mean error 0.0439306378365
best params {'n_layers': 300, 'prob': 0.9, 'error': 0.013487470149993897, 'reg_rate': 0.0001}
({'n_layers': 300, 'prob': 0.9, 'error': 0.013487470149993897, 'reg_rate': 0.0001})
Test result: 0.044186055660247803

